In [2]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain.embeddings import (
    HuggingFaceEmbeddings,
)
os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-03-01-preview"
# embeddings = AzureOpenAIEmbeddings(
#     model="text-embedding-3-small",
# )

embeddings = HuggingFaceEmbeddings(
                model_name="BAAI/bge-large-en-v1.5",
                # model_kwargs={'device': 'cpu'},
                encode_kwargs={'normalize_embeddings': False},
            )
from langchain.storage import LocalFileStore, RedisStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import FAISS
store = RedisStore(redis_url="redis://localhost:6379")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
embeddings, store, namespace="bge-large"
)
row_string = []
with open('./data/clean_relations', 'r') as f:
    r_data = [line.strip() for line in f]
all_db = FAISS.from_texts(r_data, cached_embedder)
# retriever = db.as_retriever(search_kwargs={"k": 5})

/tmp/ipykernel_972831/2109291646.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/media/disk1/chatgpt/miniconda3/envs/self-rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Run_long_form answer

In [3]:
from vllm import LLM, SamplingParams
# model = LLM(model='/media/disk2/llama_factory/generation_1124_special', trust_remote_code=True, tensor_parallel_size=4)
# model = LLM(model='/media/disk2/llama_factory/generation_1209_no_mask', trust_remote_code=True, tensor_parallel_size=1)
# model = LLM(model='/media/disk1/chatgpt/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920', trust_remote_code=True, tensor_parallel_size=1)

model = LLM(model='/media/disk2/llama_factory/generation_0122_no_mask', trust_remote_code=True, tensor_parallel_size=1)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/media/disk2/llama_factory/generation_0122_no_mask')


WARNING 01-27 02:05:05 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.


2025-01-27 02:05:05,677	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 01-27 02:05:09 config.py:905] Defaulting to use mp for distributed inference
INFO 01-27 02:05:09 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/media/disk2/llama_factory/generation_0122_no_mask', speculative_config=None, tokenizer='/media/disk2/llama_factory/generation_0122_no_mask', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, ser

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  4.91it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:00<00:00,  3.34it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.78it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.07it/s]



(VllmWorkerProcess pid=980330) INFO 01-27 02:05:16 model_runner.py:1067] Loading model weights took 3.7420 GB
(VllmWorkerProcess pid=980331) INFO 01-27 02:05:16 model_runner.py:1067] Loading model weights took 3.7420 GB
(VllmWorkerProcess pid=980329) INFO 01-27 02:05:16 model_runner.py:1067] Loading model weights took 3.7420 GB
INFO 01-27 02:05:16 model_runner.py:1067] Loading model weights took 3.7420 GB
INFO 01-27 02:05:18 distributed_gpu_executor.py:57] # GPU blocks: 61347, # CPU blocks: 8192
INFO 01-27 02:05:18 distributed_gpu_executor.py:61] Maximum concurrency for 8192 tokens per request: 119.82x
(VllmWorkerProcess pid=980331) INFO 01-27 02:05:20 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=980331) INFO 01-27 02:05:20 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memo

In [4]:
#是否带special token 分开传入，用于检索和生成
sampling_params = SamplingParams(
            temperature=0.0, top_p=1.0,max_tokens=100, logprobs=5, skip_special_tokens=False, include_stop_str_in_output=True)
PROMPT_DICT = {"llama3": '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'}
test = model.generate([PROMPT_DICT["llama3"].format(input="what all does google now do?")], sampling_params)[0].outputs[0]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.05s/it, est. speed input: 16.22 toks/s, output: 95.44 toks/s]


In [5]:
def load_special_tokens(tokenizer, use_grounding=False, use_utility=False):
    rel_tokens = {}
    for token in ['[Unrelevant]','[Partially Relevant]','[Fully Relevant]']:
        rel_tokens[token] = tokenizer.convert_tokens_to_ids(token)
    reason_tokens = {}
    for token in ['[Fully Reasonable]', '[Partially Reasonable]', '[Unreasonable]']:
        reason_tokens[token] = tokenizer.convert_tokens_to_ids(token)
    # ut_tokens = None
    # if use_utility is True:
    #     ut_tokens = {}
    #     for token in utility_tokens_names:
    #         ut_tokens[token] = tokenizer.convert_tokens_to_ids(token)

    return rel_tokens, reason_tokens
rel_tokens, reason_tokens = load_special_tokens(tokenizer)

In [ ]:

import json
with open('/media/disk1/chatgpt/zh/ToG/data/graliqa.json', 'r',encoding='utf-8') as f:
    test_data = json.load(f)
print(len(test_data))

In [6]:
import json
with open('./data/merged/WebQSP_test.json', 'r',encoding='utf-8') as f:
    test_data = json.load(f)
print(len(test_data))

1639


In [7]:
import json
with open('./data/merged/CWQ_test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
print(len(test_data))

3531


### Tree fix

In [7]:
from typing import List
import random
def save_to_json(data: List, data_path=''):
    if not os.path.isfile(data_path):
        # 文件不存在，创建新列表并写入文件
        with open(data_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        return
    try:
        # 尝试读取现有文件
        with open(data_path, 'r', encoding='utf-8') as file:
            # 加载现有的JSON数据
            existing_data = json.load(file)
            existing_data.extend(data)
    except json.JSONDecodeError:
        # 文件不是有效的JSON，打印错误信息并退出
        print(f"文件 {data_path} 不是有效的JSON格式。")
        return
    except ValueError as e:
        # 打印错误信息并退出
        print(e)
        return
    # 将更新后的数据写回文件
    with open(data_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)
def random_sample(lst, k=3):
    try:
        # 尝试从列表中随机抽取k个不重复的元素
        return random.sample(lst, k)
    except ValueError:
        # 如果列表长度小于k，返回整个列表
        return lst

In [28]:
import numpy as np
import re
from src.sparql_utils import *
import random
#维护一个relation 池
def run_relation_generation_batch(model, prompt, new_retrieval, context, topic_entity, hypo=True, use_1hop=True, income=None):
    # 初始化关系得分字典、最终预测列表、总体得分列表、最终上下文列表、段落集合
    rel_score_dict = {}
    final_preds = []
    overall_scores = []
    final_contexts = []
    paragraph = set()
    # 如果是新检索，则设置检索标记为"[Retrieve Relation]"，否则设置为"[Retrieve Relation]"
    if new_retrieval:
        retrieval_token = "[Retrieve Relation]"
    else:
        retrieval_token = "[Retrieve Relation]"
    # 如果使用1跳关系，则获取1跳关系，否则使用所有关系
    if use_1hop:
        candidate_relations = []
        for entity in topic_entity:
            try:
                candidate_relations.extend(get_1hop_relations_with_odbc(entity))
            except:
                continue
        if len(candidate_relations):
            vec_db = FAISS.from_texts(candidate_relations, cached_embedder)
        else:
            vec_db = all_db
        retriever = vec_db.as_retriever(search_kwargs={"k": 5})
    else:
        retriever = all_db.as_retriever(search_kwargs={"k": 5})
    # 获取段落内容
    paragraph.update([page.page_content.strip() for page in retriever.invoke(prompt.split('\n\n')[1].split('<|eot_id|>')[0] + ' '+ context)])
    # 如果使用假设，则生成假设关系
    if hypo:
        hypo_rel = model.generate(prompt + retrieval_token, sampling_params)[0].outputs[0].text
        pattern = r'(\w+\.\w+\.\w+)\[(.*?)\]'
        if '[Retrieve Entity]' in hypo_rel:
            hypo_rel = hypo_rel.split('[Retrieve Entity]')[0]
        matches =  dict(re.findall(pattern, hypo_rel))
        string = ''
        for k,v in matches.items():
            if v in ['Fully Relevant', 'Partially Relevant']:
                string += k + ' '
        paragraph.update([page.page_content.strip() for page in retriever.invoke(string)])
    paragraph.discard(income)
    aug_prompts =  ["<paragraph>{}</paragraph>".format(';'.join(p))  for p in [list(paragraph)[i: i+5] for i in range(0, len(paragraph), 5)]]
    
    preds = model.generate([prompt + retrieval_token + aug for aug in aug_prompts], sampling_params)
    for p_id, pred in enumerate(preds):
        pred_token_ids = pred.outputs[0].token_ids
        pred_text_1 = pred.outputs[0].text
        pred_log_probs = pred.outputs[0].logprobs
        seq_score = pred.outputs[0].cumulative_logprob / \
            max(len(pred.outputs[0].token_ids), 1)
        relevance_indices = []
        for tok_idx, tok in enumerate(pred_token_ids):
            if tok in rel_tokens.values():
                relevance_indices.append(tok_idx)
        if len(relevance_indices) > 0:
            for idx in relevance_indices:
                for token, token_id in rel_tokens.items():
                    prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
                    rel_score_dict[token] = np.exp(prob)
        relevance_score = rel_score_dict['[Fully Relevant]']+ rel_score_dict['[Partially Relevant]'] / np.sum(list(rel_score_dict.values()))
        if '[Retrieve Entity]' in pred_text_1:
            processed_pred = pred_text_1.split('[Retrieve Entity]')[0] + '[Retrieve Entity]'
        else:
            processed_pred = pred_text_1
        final_preds.append(retrieval_token + aug_prompts[p_id] + processed_pred)
        overall_scores.append(0)
        final_contexts.append(pred_text_1.split('[Retrieve Entity]')[0])

    return final_preds, overall_scores, final_contexts

import re
import random
import numpy as np

def run_entity_generation_batch(model, prompt, topic_entities, context):
    # 初始化变量
    final_predictions, final_entities, final_contexts = [], [], []
    entity_prompts, income_rel = [], []
    overall_scores, name_to_id = {}, {}
    effective_count = 0

    # 提取上下文中的实体和相关性匹配
    context_pattern = r'(.+?)\[(.*?)\]'
    context_matches = dict(re.findall(context_pattern, context))

    # 构建实体提示
    for entity in topic_entities:
        for key, relevance in context_matches.items():
            if relevance in ['Fully Relevant', 'Partially Relevant']:
                try:
                    related_entities = get_another_entity(entity, key, return_label=True)
                except Exception:
                    related_entities = {}

                if related_entities:
                    # 初始化分数和实体映射
                    income_rel.append(key)
                    name_to_id[effective_count] = related_entities
                    overall_scores[effective_count] = {
                        'r_relevance': 1.0 if relevance == 'Fully Relevant' else 0.5
                    }
                    # 构造实体提示
                    entities = [
                        f'({get_label(entity)}, {key}, {rel_entity})'
                        for rel_entity in related_entities.keys()
                    ]
                    overall_scores[effective_count]['path'] = entities[:5]
                    entity_prompts.append(
                        f"<paragraph>{';'.join(entities[:5])}</paragraph>"
                    )
                    effective_count += 1

    # 生成模型预测
    augmented_prompts = [
        f"{prompt}[Retrieve Entity]{entity_prompt}"
        for entity_prompt in entity_prompts
    ]
    predictions = model.generate(augmented_prompts, sampling_params)

    # 解析预测结果
    for idx, prediction in enumerate(predictions):
        pred_output = prediction.outputs[0]
        pred_text = pred_output.text
        pred_log_probs = pred_output.logprobs
        pred_token_ids = pred_output.token_ids
        seq_score = pred_output.cumulative_logprob / max(len(pred_token_ids), 1)

        processed_pred, rel_score, reason_score, return_entities, matches = process_prediction(
            idx, pred_text, context_pattern, name_to_id, overall_scores
        )

        # 更新分数和最终结果
        overall_scores[idx].update({
            "r_match": {income_rel[idx]: context_matches[income_rel[idx]]},
            "e_match": matches,
            "reason_score": reason_score,
            "final_score": np.exp(rel_score) * np.exp(reason_score) * np.exp(overall_scores[idx]['r_relevance'])
        })

        final_predictions.append(f"[Retrieve Entity]{entity_prompts[idx]}{processed_pred}")
        final_entities.append(list(return_entities.values()))
        final_contexts.append(' '.join(return_entities.keys()))

    # 返回最终结果
    return final_predictions, [
        overall_scores[idx]['final_score'] for idx in overall_scores
    ], final_entities, final_contexts, overall_scores, income_rel


def process_prediction(idx, pred_text, pattern, name_to_id, overall_scores):
    """处理单条预测结果，计算相关性分数和返回实体。"""
    rel_score = 0
    reason_score = 0
    return_entities = {}
    count = 0

    if '[Retrieve Relation]' in pred_text:
        processed_pred = pred_text.split('[Retrieve Relation]')[0]
        matches = dict(re.findall(pattern, processed_pred))
        for key, relevance in matches.items():
            if relevance in ['Fully Relevant', 'Partially Relevant']:
                if key in name_to_id[idx]:
                    return_entities[key] = name_to_id[idx][key]
                elif key == 'Unknown Entity':
                    random_key = random.choice(list(name_to_id[idx].keys()))
                    return_entities[random_key] = name_to_id[idx][random_key]
                # 更新相关性分数
                rel_score = ((1 if relevance == 'Fully Relevant' else 0.5) +
                             count * rel_score) / (count + 1)
                count += 1
        processed_pred += '[Retrieve Relation]'

    elif '[No Retrieval]' in pred_text:
        processed_pred = pred_text
        matches = dict(re.findall(pattern, pred_text.split('[No Retrieval]')[0]))
        for key, relevance in matches.items():
            if relevance in ['Fully Relevant', 'Partially Relevant']:
                rel_score = ((1 if relevance == 'Fully Relevant' else 0.5) +
                             count * rel_score) / (count + 1)
                count += 1
    else:
        matches = dict()
        processed_pred = '[No Retrieval]Answer: None'
        rel_score = 0

    # 处理理由分数
    if '[Fully Reasonable]' in processed_pred:
        reason_score += 1
    elif '[Partially Reasonable]' in processed_pred:
        reason_score += 0.5

    return processed_pred, rel_score, reason_score, return_entities, matches
 
        

In [ ]:
# add reasonable score

count = 0
correct_ids = []
logging_res = []
for index in range(200, len(test_data)):
    hit = 0
    print(f'Process {index}')
    data_input = test_data[index]['question']
    prompt = PROMPT_DICT['llama3'].format(input= data_input)
    max_depth = 5
    # topic_entity = list(test_data[index]['topic_entity'].keys())
    topic_entity = list(test_data[index]['gold_entity_map'].keys())
    curr_depth = 1
    node_id = 0
    prediction_tree = {
        node_id: {
            "prompt": prompt,
            "pred": "[Retrieve Relation]",
            "processed_pred": "",
            "score": None,
            "topic_entity": topic_entity,
            "parent": None,
            "context": "",
            "income": ""
        }
    }
    levels = {0: [node_id]}
    while curr_depth < max_depth:
        levels[curr_depth] = []
        if curr_depth-1 in levels:
            for node in levels[curr_depth-1]:
                curr_pred = prediction_tree[node]["pred"]
                if "<|eot_id|>" in curr_pred or 'No Retrieval' in curr_pred:
                    continue
                cur_prompt = prediction_tree[node]["prompt"] + prediction_tree[node]["processed_pred"]
                score = prediction_tree[node]["score"] or 0
                topic_entity = prediction_tree[node]["topic_entity"]
                context = prediction_tree[node]["context"]
                income_rel = prediction_tree[node]["income"]
                if "Retrieve Entity" in curr_pred.split('[')[-1]:
                    retrieval_results = {}
                    preds, scores, next_entities, contexts, overall_scores, income_rel = run_entity_generation_batch(
                        model, cur_prompt, topic_entity, context)
                    for i, (pred, p_score,next_topic, context, rel) in enumerate(zip(preds, scores, next_entities, contexts,  income_rel)):
                        retrieval_results[i] = {
                            "pred": pred, "score": p_score, "next_topic": next_topic, "context": context, "income": rel, "verbose": {"path": overall_scores[i]['path'], "e_match": overall_scores[i]['e_match'], "r_match": overall_scores[i]['r_match'], 'reason_score': overall_scores[i]['reason_score']}}
                    for i, result in retrieval_results.items():
                        node_id += 1
                        node_score = (result["score"] + score) / (curr_depth // 2)
                        pred = result["pred"]
                        next_entity = result['next_topic']
                        if len(next_entity) == 0:
                            next_entity = topic_entity
                        prediction_tree[node_id] = {"prompt": cur_prompt, "pred": pred, "context": result['context'],
                                                    "score": node_score, "parent": node,
                                                    "topic_entity": next_entity, "income": result['income'], "verbose": result['verbose']}
                        if "[Continue to Retrieve Evidence]" in pred:
                            gen_result_index = pred.index("[Continue to Retrieve Evidence]")
                            prev_generation = pred[:gen_result_index]
                        else:
                            prev_generation = pred
                        prediction_tree[node_id]["processed_pred"] = prev_generation
                        levels[curr_depth].append(node_id)
                #存在前后逻辑粘连   
                if "Retrieve Relation" in curr_pred.split('[')[-1]:
                    retrieval_results = {}
                    preds, scores, contexts = run_relation_generation_batch(
                        model, cur_prompt, new_retrieval=True if ("[New Retrieval]" in curr_pred) else False, context=context, topic_entity=topic_entity, hypo=True, income=income_rel)
                    for i, (pred, p_score, context) in enumerate(zip(preds, scores, contexts)):
                        retrieval_results[i] = {
                            "pred": pred, "score": p_score, "context": context}

                    for i, result in retrieval_results.items():
                        node_id += 1
                        #计算score
                        node_score = result["score"] + score if score is not None else result["score"]
                        # node_score = result['score']
                        pred = result["pred"]
                        context = result["context"]
                        prediction_tree[node_id] = {"prompt": cur_prompt, "pred": pred,
                                                    "score": node_score, "parent": node,
                                                "topic_entity": topic_entity, "context": context, 'income': income_rel}
                        if "[Retrieve Entity]" in pred:
                            gen_result_index = pred.index("[Retrieve Entity]")
                            prev_generation = pred[:gen_result_index]
                        else:
                            prev_generation = pred
                        prediction_tree[node_id]["processed_pred"] = prev_generation
                        levels[curr_depth].append(node_id)
            current_rank = levels[curr_depth]
            node2score = {
                node_id: prediction_tree[node_id]["score"] for node_id in current_rank}
            top_nodes = sorted(node2score.items(), key=lambda x: x[1], reverse=True)[
                :3]
            levels[curr_depth] = [node[0] for node in top_nodes]
            curr_depth += 1
        else:
            break
    labels = [get_label(ans) if ans.startswith('m.') else ans for ans in test_data[index]['answer']]
    # labels = [ans['entity_name'] for ans in test_data[index]['answer']]
    # print(labels)
    max_score = 0
    for ind, tree_node in enumerate(prediction_tree.values()):
        if 'Answer' in tree_node['processed_pred']:
            if tree_node['score'] > max_score:
                max_score = tree_node['score']
                answer = tree_node['processed_pred'].split('Answer:')[-1]
    # for tree_node in prediction_tree.values():
    #     if 'Answer' in tree_node['processed_pred']:
    #         answer = tree_node['processed_pred'].split('Answer:')[-1]
    for label in labels:
        if label and label in answer:
            hit = 1
    if hit == 1:
        print('Correct')
        count += 1
        correct_ids.append(index)
    break
    # else:
        # break

        # logging_res.append({"index": index, "tree": prediction_tree})
    # if len(logging_res) == 20:
        # save_to_json(logging_res, './output/inference/cwq_test_res_1217.json')
    #     logging_res = []
#1217修改了relation-1hop,修改了score
# save_to_json(logging_res, './output/inference/cwq_test_res_1217.json')
    #注意有value标签, e.g. WebQTest-31
    # for label in labels:
    #     if label and label in prediction_tree[len(prediction_tree)-1]['processed_pred']:
    #         count += 1
    #         break
    # except:
    #     print(f'{index} Error')
    #     continue


Process 200


Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  4.33it/s, est. speed input: 623.54 toks/s, output: 168.24 toks/s]


In [35]:
for tree_node in prediction_tree.values():
    if 'Answer' in tree_node['processed_pred']:
        answer = tree_node['processed_pred'].split('Answer:')[-1]
        print(tree_node['score'], answer)

2.7182818284590455  Brussels<|eot_id|>
5.935372584634358  Brussels<|eot_id|>


In [12]:
max_score = 0
for ind, tree_node in enumerate(prediction_tree.values()):
    if 'Answer' in tree_node['processed_pred']:
        if tree_node['score'] > max_score:
            max_score = tree_node['score']
            answer = tree_node['processed_pred'].split('Answer:')[-1]
        # print(tree_node['score'])
        # print(answer)

In [18]:
answer

' Germanic peoples<|eot_id|>'

In [16]:
labels

['Belgium',
 'Brazil',
 'United States of America',
 'Australia',
 'Canada',
 'France',
 'South Africa']

In [17]:
prediction_tree

{0: {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nwhere did flemish people come from<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[Retrieve Relation]',
  'processed_pred': '',
  'score': None,
  'topic_entity': ['m.018hlv'],
  'parent': None,
  'context': '',
  'income': ''},
 1: {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nwhere did flemish people come from<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[Retrieve Relation]<paragraph>people.ethnicity.people;people.ethnicity.population;people.ethnicity.included_in_group;people.ethnicity.geographic_distribution;people.ethnicity.includes_groups</paragraph>people.ethnicity.people[Partially Relevant]people.ethnicity.population[Partially Relevant]people.ethnicity.included_in_group[Partially Relevant]people.ethnicity.geographic_distribution[Fully Relevant]people.ethnicity.includes_groups[Partially Relevant][Retrieve Entity]',
  'score': 0,
  'pa

In [19]:
end_nodes = []
for n_ind, node in prediction_tree.items():
    if 'Answer' in node['processed_pred']:
        end_nodes.append(n_ind)

queues = []
for n_ind in end_nodes:

    queue = []
    node = prediction_tree[n_ind]
    answer = node['processed_pred'].split('Answer:')[-1]
    score =  node['score']
    while node:
        parent = node['parent']
        if 'verbose' in node:
            queue.append(node['verbose'])
        if parent == None:
            queues.append({'verbose': queue, "answer": answer, "score": score})

            break
        node = prediction_tree[parent]


In [22]:
queues.sort(key=lambda x: x['score'], reverse=True)


In [26]:
queues[:10]

[{'verbose': [{'path': ['(Flemish people, people.ethnicity.included_in_group, Germanic peoples)'],
    'e_match': {'Germanic peoples': 'Partially Relevant'},
    'r_match': {'people.ethnicity.included_in_group': 'Partially Relevant'},
    'reason_score': 0.5}],
  'answer': ' Germanic peoples<|eot_id|>',
  'score': 4.481689070338065},
 {'verbose': [{'path': ['(Flemish people, people.ethnicity.includes_groups, Germanic peoples)'],
    'e_match': {'Germanic peoples': 'Partially Relevant'},
    'r_match': {'people.ethnicity.includes_groups': 'Partially Relevant'},
    'reason_score': 0.5}],
  'answer': ' Germanic peoples<|eot_id|>',
  'score': 4.481689070338065},
 {'verbose': [{'path': ['(Flemish people, people.ethnicity.includes_groups, Germanic peoples)'],
    'e_match': {'Germanic peoples': 'Partially Relevant'},
    'r_match': {'people.ethnicity.includes_groups': 'Partially Relevant'},
    'reason_score': 0.5}],
  'answer': ' Germanic peoples<|eot_id|>',
  'score': 4.481689070338065},


### Evaluate result

In [11]:
with open('/media/disk1/chatgpt/zh/graph_data/output/inference/webqsp_0122model_code_0123_res.json', 'r', encoding='utf-8') as f:
    logging_res_0123 = json.load(f)

In [13]:
count = 0

for index in range(200, 300):
    hit_1 = 0
    hit_2 = 0
    max_score = 0
    labels = [get_label(ans) if ans.startswith('m.') else ans for ans in test_data[index]['answer']]
    for ind, tree_node in enumerate(logging_res_0123[index]['tree'].values()):
        if 'Answer' in tree_node['processed_pred']:
            if tree_node['score'] > max_score:
                max_score = tree_node['score']
                answer_1 = tree_node['processed_pred'].split('Answer:')[-1]
            answer_2 = tree_node['processed_pred'].split('Answer:')[-1]
            for label in labels:
                if label in answer_2:
                    hit_2 = 1
    for label in labels:
        if label in answer_1:
            hit_1 = 1
    if hit_1 == 0 and hit_2 == 1:
        print(index)




200
207
218
219
222
228
241
249
251
264
277
286


In [14]:
print(labels)
logging_res[106]['tree']

['Comedian', 'Actor', 'Television producer', 'Singer', 'Model']


{'0': {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nwhat was lucille ball<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[Retrieve Relation]',
  'processed_pred': '',
  'score': None,
  'topic_entity': ['m.04nw9'],
  'parent': None,
  'context': ''},
 '1': {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nwhat was lucille ball<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[Retrieve Relation]<paragraph>base.americancomedy.double_act.comic;people.person.places_lived;people.person.profession;base.popstra.celebrity.religion;people.person.religion</paragraph>base.americancomedy.double_act.comic[Partially Relevant]people.person.places_lived[Unrelevant]people.person.profession[Partially Relevant]base.popstra.celebrity.religion[Unrelevant]people.person.religion[Unrelevant][Retrieve Entity]',
  'score': 0,
  'parent': 0,
  'topic_entity': ['m.04nw9'],
  'context': 'base.americancomedy.double_act.comic[P

In [ ]:
end_nodes = []
for n_ind, node in prediction_tree.values():
    if 'Answer' in node['processed_pred']:
        end_nodes.append(n_ind)
for n_ind, node in end_nodes:
    while node != 0:
        parent = prediction_tree[node]['parent']
        



In [89]:
def backtracking_prediction_tree(levels: dict[int,list[int]], curr_depth: int, prediction_tree: dict[int, dict]) -> dict[int,list[int]]:
    '''
    get best tracking from prediction_tree base on levels
    '''
    parent = 0 
    best_selections = {}
    # Traverse from the bottom 
    levels = {k: v for k, v in levels.items() if len(v) > 0 and k != 0} # remove empty list in levels
    
    for path_i, node in enumerate(levels[len(levels)]): # beam search 
        if node == 0:
            break
        best_selections[path_i] = [node] 
        current_node = node 
        current_level = curr_depth 
        if current_node is None:
            continue
        while current_level > 0 and current_node is not None:
            parent = prediction_tree[current_node]["parent"]
            best_selections[path_i] = [parent] + best_selections[path_i] 
            current_node = parent 
            current_level -= 1
    return best_selections
backtracking_prediction_tree(levels, 5, prediction_tree)

{0: [None, 0, 2, 3], 1: [None, 0, 2, 4], 2: [None, 0, 1, 5]}

In [ ]:
def cal_eval_metric(best_pred, preds, answers):
    correct, total = 0.0, 0.0
    for entity in preds:
        if entity in answers:
            correct += 1
        total += 1
    if len(answers) == 0:
        if total == 0:
            return 1.0, 1.0, 1.0, 1.0 # precision, recall, f1, hits
        else:
            return 0.0, 1.0, 0.0, 1.0 # precision, recall, f1, hits
    else:
        hits = float(best_pred in answers)
        if total == 0:
            return 1.0, 0.0, 0.0, hits # precision, recall, f1, hits
        else:
            precision, recall = correct / total, correct / len(answers)
            f1 = 2.0 / (1.0 / precision + 1.0 / recall) if precision != 0 and recall != 0 else 0.0
            return precision, recall, f1, hits

In [ ]:
def cal_eval_metric(preds, answers):
    correct, total = 0.0, 0.0
    for entity in preds:
        if entity in answers:
            correct += 1
        total += 1
    # if len(answers) == 0:
    #     if total == 0:
    #         return 1.0, 1.0, 1.0, 1.0 # precision, recall, f1, hits
    #     else:
    #         return 0.0, 1.0, 0.0, 1.0 # precision, recall, f1, hits
    # else:
    if total != 0:
        hits = 1
    else: 
        hits = 0
    if total == 0:
        return 1.0, 0.0, 0.0, hits # precision, recall, f1, hits
    else:
        precision, recall = correct / total, correct / len(answers)
        f1 = 2.0 / (1.0 / precision + 1.0 / recall) if precision != 0 and recall != 0 else 0.0
        return precision, recall, f1, hits

In [ ]:
cal_eval_metric(preds, answers)

In [ ]:
count = 0
correct_ids = []
logging_res = []
for index in range(0, len(test_data)):
# index = 483
    hit = 0
    print(f'Process {index}')
    data_input = test_data[index]['question']
    prompt = PROMPT_DICT['llama3'].format(input= data_input)
    max_depth = 5
    # topic_entity = list(test_data[index]['topic_entity'].keys())
    topic_entity = list(test_data[index]['gold_entity_map'].keys())
    # pred = model.generate([prompt], sampling_params)[0]
    # pred_text = pred.outputs[0].text
    # if '[New Retrieval]' in pred_text:
    curr_depth = 1
    terminated = False
    node_id = 0
    prediction_tree = {}
    levels = {}
    prediction_tree[node_id] = {"prompt": prompt, "pred": "[New Retrieval]",
                                "processed_pred": "", "score": None, "topic_entity": topic_entity, "parent": None, "context": ''}
    levels[0] = [0]
    while curr_depth < max_depth:
        levels[curr_depth] = []
        if curr_depth-1 in levels:
            for node in levels[curr_depth-1]:
                curr_pred = prediction_tree[node]["pred"]
                if "<|eot_id|>" in curr_pred:
                    continue
                prompt = prediction_tree[node]["prompt"]
                prev_generation = prediction_tree[node]["processed_pred"]
                score = prediction_tree[node]["score"]
                topic_entity = prediction_tree[node]["topic_entity"]
                context = prediction_tree[node]['context']
                cur_prompt = prompt + prev_generation
                if "Retrieve Entity" in curr_pred.split('[')[-1]:
                    retrieval_results = {}
                    preds, scores, next_entities, contexts = run_entity_generation_batch(
                        model, cur_prompt, topic_entity, context)
                    for i, (pred, p_score,next_topic, context) in enumerate(zip(preds, scores, next_entities, contexts)):
                        retrieval_results[i] = {
                            "pred": pred, "score": p_score, "next_topic": next_topic, "context": context}

                    for i, result in retrieval_results.items():
                        node_id += 1
                        node_score = result["score"] * \
                            score if score is not None else result["score"]
                        pred = result["pred"]
                        next_entity = result['next_topic']
                        if len(next_entity) == 0:
                            next_entity = topic_entity
                        prediction_tree[node_id] = {"prompt": cur_prompt, "pred": pred, "context": result['context'],
                                                    "score": node_score, "parent": node,
                                                    "topic_entity": next_entity}
                        if "[Continue to Retrieve Evidence]" in pred:
                            gen_result_index = pred.index("[Continue to Retrieve Evidence]")
                            prev_generation = pred[:gen_result_index]
                        else:
                            prev_generation = pred
                        prediction_tree[node_id]["processed_pred"] = prev_generation
                        levels[curr_depth].append(node_id)
                #存在前后逻辑粘连   
                if "New Retrieval" in curr_pred.split('[')[-1] or "Continue to Retrieve Evidence" in curr_pred.split('[')[-1]:
                    retrieval_results = {}
                    preds, scores, contexts = run_relation_generation_batch(
                        model, cur_prompt, new_retrieval=True if ("[New Retrieval]" in curr_pred) else False, context=context, topic_entity=topic_entity, hypo=True)
                    for i, (pred, p_score, context) in enumerate(zip(preds, scores, contexts)):
                        retrieval_results[i] = {
                            "pred": pred, "score": p_score, "context": context}

                    for i, result in retrieval_results.items():
                        node_id += 1
                        node_score = result["score"] * \
                            score if score is not None else result["score"]
                        pred = result["pred"]
                        context = result["context"]
                        prediction_tree[node_id] = {"prompt": cur_prompt, "pred": pred,
                                                    "score": node_score, "parent": node,
                                                    "topic_entity": topic_entity, "context": context}
                        if "[Retrieve Entity]" in pred:
                            gen_result_index = pred.index("[Retrieve Entity]")
                            prev_generation = pred[:gen_result_index]
                        else:
                            prev_generation = pred
                        prediction_tree[node_id]["processed_pred"] = prev_generation
                        levels[curr_depth].append(node_id)
            current_rank = levels[curr_depth]
            node2score = {
                node_id: prediction_tree[node_id]["score"] for node_id in current_rank}
            top_nodes = sorted(node2score.items(), key=lambda x: x[1], reverse=True)[
                :3]
            levels[curr_depth] = [node[0] for node in top_nodes]
            curr_depth += 1
        else:
            break
    labels = [get_label(ans) if ans.startswith('m.') else ans for ans in test_data[index]['answer']]
    # labels = [ans['entity_name'] for ans in test_data[index]['answer']]
    # print(labels)
    for tree_node in prediction_tree.values():
        if 'Answer' in tree_node['processed_pred']:
            answer = tree_node['processed_pred'].split('Answer:')[-1]
            for label in labels:
                if label and label in answer:
                    hit = 1
    if hit == 1:
        print('Correct')
        count += 1
        correct_ids.append(index)
        break
    # else:
        # break

        # logging_res.append({"index": index, "tree": prediction_tree})
    # if len(logging_res) == 20:
        # save_to_json(logging_res, './output/inference/cwq_test_res_1217.json')
    #     logging_res = []
#1217修改了relation-1hop,修改了score
# save_to_json(logging_res, './output/inference/cwq_test_res_1217.json')
    #注意有value标签, e.g. WebQTest-31
    # for label in labels:
    #     if label and label in prediction_tree[len(prediction_tree)-1]['processed_pred']:
    #         count += 1
    #         break
    # except:
    #     print(f'{index} Error')
    #     continue


#### Test Part

In [1]:
import json
with open('./output/inference/cwq_test_res_1217.json', 'r', encoding='utf-8') as f:
    cwq_result = json.load(f)

In [ ]:
for i, error in enumerate(result):
    if error['index'] == 14:
        print(i, error)

In [ ]:
cwq_result[18]

In [ ]:
get_another_entity('m.0y496z9', 'location.location.time_zones')